In [72]:
#imports
import pandas as pd
from pathlib import Path
from df_functions import change_colnames, change_organism_ID
import json
import numpy as np
from collections import Counter
pd.set_option('display.max_columns', 114)

In [73]:
#data
datadir = Path('data')
receptorfull = 'receptor_full_v3(14April).csv'
lipopoly = 'db_lipopoly_KABAT.csv'

In [74]:
receptor = pd.read_csv(datadir / receptorfull, low_memory=False)
receptor.drop_duplicates(inplace=True)

lipopoly = pd.read_csv(datadir / lipopoly, low_memory=False)
lipopoly.drop_duplicates(inplace=True)

In [75]:
receptor.shape

(34808, 71)

In [76]:
receptor.rename(columns= lambda c: change_colnames(c), inplace=True)

In [77]:
#dict with organism names and their corresponding number
path_to_dict = 'organism_dict.json'
with open(path_to_dict, 'r') as od:
    organism_dict = json.load(od)

change_organism_ID(data=receptor, organism_dict=organism_dict)

In [78]:
receptor.head(10)

,Group Receptor ID,Receptor ID,Reference IRI,Epitope IRI,Epitope,Antigen,Organism,Response Type,Assay IDs,Reference Name,STABLE ID,Synonyms,Receptor Type,Heavy Type,Heavy Species,Heavy_Org,Heavy Nucleotide,Curated Heavy V Gene,Calculated Heavy V Gene,Curated Heavy D Gene,Calculated Heavy D Gene,Curated Heavy J Gene,Calculated Heavy J Gene,Heavy Full Sequence,Heavy Accession,Heavy CDR3 Curated,Heavy CDR3 Calculated,Heavy CDR3 Start Curated,Heavy CDR3 End Curated,Heavy CDR3 Start Calculated,Heavy CDR3 End Calculated,Heavy CDR1 Curated,Heavy CDR1 Calculated,Heavy CDR1 Start Curated,Heavy CDR1 End Curated,Heavy CDR1 Start Calculated,Heavy CDR1 End Calculated,Heavy CDR2 Curated,Heavy CDR2 Calculated,Heavy CDR2 Start Curated,Heavy CDR2 End Curated,Heavy CDR2 Start Calculated,Heavy CDR2 End Calculated,Light Type,Light Species,Light_Org,Light Nucleotide,Curated Light V Gene,Calculated Light V Gene,Curated Light D Gene,Calculated Light D Gene,Curated Light J Gene,Calculated Light J Gene,Light Full Sequence,Light Accession,Light CDR3 Curated,Light CDR3 Calculated,Light CDR3 Start Curated,Light CDR3 End Curated,Light CDR3 Start Calculated,Light CDR3 End Calculated,Light CDR1 Curated,Light CDR1 Calculated,Light CDR1 Start Curated,Light CDR1 End Curated,Light CDR1 Start Calculated,Light CDR1 End Calculated,Light CDR2 Curated,Light CDR2 Calculated,Light CDR2 Start Curated,Light CDR2 End Curated,Light CDR2 Start Calculated,Light CDR2 End Calculated
0,1,1,http://www.iedb.org/reference/279,http://www.iedb.org/epitope/77533,"G58, L59, S61, P62, C63, K64, D65, G109, G110,...",Angiogenin precursor,Homo sapiens,B cell,"32, 1686197",26-2F,NaN,NaN,heavylight,heavy,9606.0,Homo sapiens,NaN,NaN,IGHV5-9*01,NaN,NaN,NaN,IGHJ4*01,EVMLVESGGGLVKPGGSLKLSCAASGFTFSSYTMSWVRQTPEKRLE...,1H0D_B,NaN,TRLGDYGYAYTMDY,NaN,NaN,97.0,110.0,NaN,GFTFSSYT,NaN,NaN,26.0,33.0,NaN,ISSGGGNT,NaN,NaN,51.0,58.0,light,9606.0,Homo sapiens,NaN,NaN,IGKV3-2*01,NaN,NaN,NaN,IGKJ5*01,DIVLTQSPASLAVSLGQRATISCRASESVDNYGISFMSWFQQKPGQ...,1H0D_A,NaN,QQSKEVPLT,NaN,NaN,93.0,101.0,NaN,ESVDNYGISF,NaN,NaN,27.0,36.0,NaN,AAS,NaN,NaN,54.0,56.0
1,2,2,http://www.iedb.org/reference/280,http://www.iedb.org/epitope/190616,"S206, S207, A208, A209, T210, K211, K212, T213...",Outer surface protein A,Borreliella burgdorferi,B cell,"33, 2012290",LA-2,NaN,NaN,heavylight,heavy,10090.0,Mus musculus,NaN,NaN,IGHV9-2-1*01,NaN,NaN,NaN,IGHJ4*01,QIQLVQSGPELKKPGETVKISCKASGYTFTDYSMYWVKQAPGKGLK...,1FJ1_B,NaN,ARGLDS,NaN,NaN,97.0,102.0,NaN,GYTFTDYS,NaN,NaN,26.0,33.0,NaN,INTETGEP,NaN,NaN,51.0,58.0,light,10090.0,Mus musculus,NaN,NaN,IGKV19-93*02,NaN,NaN,NaN,IGKJ1*01,DIQMTQSPSSLSATLGGKVTITCKASQDINKYIAWYQHKPGKGPRL...,1FJ1_A,NaN,LQYDNLQRT,NaN,NaN,89.0,97.0,NaN,QDINKY,NaN,NaN,27.0,32.0,NaN,YTS,NaN,NaN,50.0,52.0
2,3,3,http://www.iedb.org/reference/310,http://www.iedb.org/epitope/122064,"E56, G68, E70, R101, G102, S103, K104, P105, D107",T-cell surface glycoprotein CD3 epsilon chain ...,Homo sapiens,B cell,"167, 1700408, 1700409, 1700410, 1700411, 17004...",OKT3,NaN,NaN,heavylight,heavy,10090.0,Mus musculus,NaN,NaN,IGHV1-4*01,NaN,NaN,NaN,IGHJ2*01,QVQLQQSGAELARPGASVKMSCKASGYTFTRYTMHWVKQRPGQGLE...,1SY6_H,NaN,ARYYDDHYCLDY,NaN,NaN,97.0,108.0,NaN,GYTFTRYT,NaN,NaN,26.0,33.0,NaN,INPSRGYT,NaN,NaN,51.0,58.0,light,10090.0,Mus musculus,NaN,NaN,IGKV4-59*01,NaN,NaN,NaN,IGKJ4*01,QIVLTQSPAIMSASPGEKVTMTCSASSSVSYMNWYQQKSGTSPKRW...,1SY6_L,NaN,QQWSSNPFT,NaN,NaN,88.0,96.0,NaN,SSVSY,NaN,NaN,27.0,31.0,NaN,DTS,NaN,NaN,49.0,51.0
3,4,4,http://www.iedb.org/reference/312,http://www.iedb.org/epitope/226962,"V: F43; W: Y71, K74, Q105, I106, M107, R108, I...",VEGF homodimer,Homo sapiens,B cell,"1708357, 1708358, 1708372, 1708373, 1708375",Fab-12 variant Y0317,NaN,NaN,heavylight,heavy,10090.0,Mus musculus,NaN,NaN,IGHV3-23*04,NaN,NaN,NaN,IGHJ5*01,EVQLVESGGGLVQPGGSLRLSCAASGYDFTHYGMNWVRQAPGKGLE...,1CZ8_H,NaN,AKYPYYYGTSHWYFDV,NaN,NaN,97.0,112.0,NaN,GYDFTHYG,NaN,NaN,26.0,33.0,NaN,INTYTGEP,NaN,NaN,51.0,58.0,light,10090.0,Mus musculus,NaN,NaN,IGKV1-16*01,NaN,NaN,NaN,IGKJ1*0

In [79]:
receptor['Response Type'].value_counts()

T cell    31429
B cell     3379
Name: Response Type, dtype: int64

In [80]:
#Filter out the B cell response
data = receptor[(receptor['Response Type'] == 'B cell') & 
                ((receptor['Heavy_Org'] == 'Homo sapiens') |
                (receptor['Heavy_Org'] == 'Mus musculus'))]
#data = receptor[receptor['Response Type'] == 'B cell']
data['Heavy_Org'].value_counts()

Mus musculus    1390
Homo sapiens    1287
Name: Heavy_Org, dtype: int64

In [81]:
#Insert new column so all CDRs sequences are in the same column instead of 2 (Curated and Calculated).
data.insert(26, "HCDR3", data['Heavy CDR3 Curated'].fillna(data['Heavy CDR3 Calculated']))
data.insert(26, "HCDR2", data['Heavy CDR2 Curated'].fillna(data['Heavy CDR2 Calculated']))
data.insert(26, "HCDR1", data['Heavy CDR1 Curated'].fillna(data['Heavy CDR1 Calculated']))


data.insert(26, "LCDR3", data['Light CDR3 Curated'].fillna(data['Light CDR3 Calculated']))
data.insert(26, "LCDR2", data['Light CDR2 Curated'].fillna(data['Light CDR2 Calculated']))
data.insert(26, "LCDR1", data['Light CDR1 Curated'].fillna(data['Light CDR1 Calculated']))

#Do the same for V gene and J gene
data.insert(18, "Vgene", data['Curated Heavy V Gene'].fillna(data['Calculated Heavy V Gene']))
data.insert(22, "Jgene", data['Curated Heavy J Gene'].fillna(data['Calculated Heavy J Gene']))
data.shape

(2677, 81)

In [82]:
s = 'ARGLDS'
s[2:]

'GLDS'

In [83]:
#Change to Kabat numbering
data.dropna(subset=['HCDR3'], inplace=True)
data['HCDR3'] = data['HCDR3'].apply(lambda x: x[2:])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [84]:
cdr3_grouped = data.groupby('Epitope')['HCDR3'].value_counts()
unique, counts = np.unique(cdr3_grouped.values, return_counts=True)
#print(np.asarray((unique, counts)).T)
#data['HCDR3'].value_counts()

In [85]:
#concatenate the lipopoly db to the IEDB db on 3 columns (only for CDR3)
lipopoly.rename({'CDR3':'HCDR3', 'antigen_disease':'Antigen'},axis='columns', inplace=True)
lipopoly['Antigen'] = 'lipopolysaccharide_own'
lipopoly['Epitope'] = 'lipopolysaccharide_own'

lipo_cols = ['HCDR3', 'Antigen', 'Epitope']
lipopoly = lipopoly.filter(lipo_cols, axis='columns')
lipopoly.dropna(subset=['HCDR3'], inplace=True) #drop duplicates

data = pd.concat([data, lipopoly], ignore_index=True, axis=0, sort=False)

In [90]:
data['HCDR3']

0             LGDYGYAYTMDY
1                     GLDS
2               YYDDHYCLDY
3           YPYYYGTSHWYFDV
4           YPYYYGTSHWYFDV
5             RAPFYGNHAMDY
6              MWYYGTYYFDY
7              MWYYGTYYFDY
8              MWYYGTYYFDY
9              MWYYGTYYFDY
10             MWYYGTYYFDY
11             MWYYGTYYFDY
12             MWYYGTYYFDY
13             MWYYGTYYFDY
14             MWYYGTYYFDY
15             MWYYGTYYFDY
16             MWYYGTYYFDY
17             MWYYGTYYFDY
18             MWYYGTYYFDY
19             MWYYGTYYFDY
20             MWYYGTYYFDY
21             MWYYGTYYFDY
22             MWYYGTYYFDY
23             MWYYGTYYFDY
24             MWYYGTYYFDY
25             MWYYGTYYFDY
26             MWYYGTYYFDY
27             MWYYGTYYFDY
28             MWYYGTYYFDY
29             MWYYGTYYFDY
               ...        
2745            GGSGATINYW
2746            DNGVKAFDIW
2747               WGRTQDW
2748         EGAPPGKYAFDIW
2749                DDLKNW
2750              GDGYNFFW
2

In [91]:
def export_CDR_file(df, merge=False):
    '''Function that returns a dataframe where the column for the region is without NA values
    and contains no duplicates'''
    
    #regions is a list of unique CDRs
    regions = ['HCDR3', 'HCDR2', 'HCDR1', 'LCDR3', 'LCDR2', 'LCDR1']
    
    for CDR in regions:
        no_NA_df = df.dropna(subset=[CDR])
        data = no_NA_df.drop_duplicates(subset=[CDR, 'Epitope'])
        
        if CDR == 'HCDR3' and merge != False:
            data = pd.concat([data, lipopoly], ignore_index=True, axis=0, sort=False)
            print('merge=True')
            #print(data['Antigen'].value_counts())

        print(CDR, data.shape, data[CDR].isnull().sum())
        filename = 'KABAT_human_mouseBCR'+CDR+'.csv'
        export = data.to_csv(datadir / filename, index=False)
    return export

In [92]:
export_CDR_file(df=data, merge=True)

merge=True
HCDR3 (2204, 81) 0
HCDR2 (1786, 81) 0
HCDR1 (1764, 81) 0
LCDR3 (1794, 81) 0
LCDR2 (1709, 81) 0
LCDR1 (1737, 81) 0


In [71]:
data

,Group Receptor ID,Receptor ID,Reference IRI,Epitope IRI,Epitope,Antigen,Organism,Response Type,Assay IDs,Reference Name,STABLE ID,Synonyms,Receptor Type,Heavy Type,Heavy Species,Heavy_Org,Heavy Nucleotide,Curated Heavy V Gene,Vgene,Calculated Heavy V Gene,Curated Heavy D Gene,Calculated Heavy D Gene,Jgene,Curated Heavy J Gene,Calculated Heavy J Gene,Heavy Full Sequence,Heavy Accession,Heavy CDR3 Curated,LCDR1,LCDR2,LCDR3,HCDR1,HCDR2,HCDR3,Heavy CDR3 Calculated,Heavy CDR3 Start Curated,Heavy CDR3 End Curated,Heavy CDR3 Start Calculated,Heavy CDR3 End Calculated,Heavy CDR1 Curated,Heavy CDR1 Calculated,Heavy CDR1 Start Curated,Heavy CDR1 End Curated,Heavy CDR1 Start Calculated,Heavy CDR1 End Calculated,Heavy CDR2 Curated,Heavy CDR2 Calculated,Heavy CDR2 Start Curated,Heavy CDR2 End Curated,Heavy CDR2 Start Calculated,Heavy CDR2 End Calculated,Light Type,Light Species,Light_Org,Light Nucleotide,Curated Light V Gene,Calculated Light V Gene,Curated Light D Gene,Calculated Light D Gene,Curated Light J Gene,Calculated Light J Gene,Light Full Sequence,Light Accession,Light CDR3 Curated,Light CDR3 Calculated,Light CDR3 Start Curated,Light CDR3 End Curated,Light CDR3 Start Calculated,Light CDR3 End Calculated,Light CDR1 Curated,Light CDR1 Calculated,Light CDR1 Start Curated,Light CDR1 End Curated,Light CDR1 Start Calculated,Light CDR1 End Calculated,Light CDR2 Curated,Light CDR2 Calculated,Light CDR2 Start Curated,Light CDR2 End Curated,Light CDR2 Start Calculated,Light CDR2 End Calculated
0,1.0,1.0,http://www.iedb.org/reference/279,http://www.iedb.org/epitope/77533,"G58, L59, S61, P62, C63, K64, D65, G109, G110,...",Angiogenin precursor,Homo sapiens,B cell,"32, 1686197",26-2F,NaN,NaN,heavylight,heavy,9606.0,Homo sapiens,NaN,NaN,IGHV5-9*01,IGHV5-9*01,NaN,NaN,IGHJ4*01,NaN,IGHJ4*01,EVMLVESGGGLVKPGGSLKLSCAASGFTFSSYTMSWVRQTPEKRLE...,1H0D_B,NaN,ESVDNYGISF,AAS,QQSKEVPLT,GFTFSSYT,ISSGGGNT,LGDYGYAYTMDY,TRLGDYGYAYTMDY,NaN,NaN,97.0,110.0,NaN,GFTFSSYT,NaN,NaN,26.0,33.0,NaN,ISSGGGNT,NaN,NaN,51.0,58.0,light,9606.0,Homo sapiens,NaN,NaN,IGKV3-2*01,NaN,NaN,NaN,IGKJ5*01,DIVLTQSPASLAVSLGQRATISCRASESVDNYGISFMSWFQQKPGQ...,1H0D_A,NaN,QQSKEVPLT,NaN,NaN,93.0,101.0,NaN,ESVDNYGISF,NaN,NaN,27.0,36.0,NaN,AAS,NaN,NaN,54.0,56.0
1,2.0,2.0,http://www.iedb.org/reference/280,http://www.iedb.org/epitope/190616,"S206, S207, A208, A209, T210, K211, K212, T213...",Outer surface protein A,Borreliella burgdorferi,B cell,"33, 2012290",LA-2,NaN,NaN,heavylight,heavy,10090.0,Mus musculus,NaN,NaN,IGHV9-2-1*01,IGHV9-2-1*01,NaN,NaN,IGHJ4*01,NaN,IGHJ4*01,QIQLVQSGPELKKPGETVKISCKASGYTFTDYSMYWVKQAPGKGLK...,1FJ1_B,NaN,QDINKY,YTS,LQYDNLQRT,GYTFTDYS,INTETGEP,GLDS,ARGLDS,NaN,NaN,97.0,102.0,NaN,GYTFTDYS,NaN,NaN,26.0,33.0,NaN,INTETGEP,NaN,NaN,51.0,58.0,light,10090.0,Mus musculus,NaN,NaN,IGKV19-93*02,NaN,NaN,NaN,IGKJ1*01,DIQMTQSPSSLSATLGGKVTITCKASQDINKYIAWYQHKPGKGPRL...,1FJ1_A,NaN,LQYDNLQRT,NaN,NaN,89.0,97.0,NaN,QDINKY,NaN,NaN,27.0,32.0,NaN,YTS,NaN,NaN,50.0,52.0
2,3.0,3.0,http://www.iedb.org/reference/310,http://www.iedb.org/epitope/122064,"E56, G68, E70, R101, G102, S103, K104, P105, D107",T-cell surface glycoprotein CD3 epsilon chain ...,Homo sapiens,B cell,"167, 1700408, 1700409, 1700410, 1700411, 17004...",OKT3,NaN,NaN,heavylight,heavy,10090.0,Mus musculus,NaN,NaN,IGHV1-4*01,IGHV1-4*01,NaN,NaN,IGHJ2*01,NaN,IGHJ2*01,QVQLQQSGAELARPGASVKMSCKASGYTFTRYTMHWVKQRPGQGLE...,1SY6_H,NaN,SSVSY,DTS,QQWSSNPFT,GYTFTRYT,INPSRGYT,YYDDHYCLDY,ARYYDDHYCLDY,NaN,NaN,97.0,108.0,NaN,GYTFTRYT,NaN,NaN,26.0,33.0,NaN,INPSRGYT,NaN,NaN,51.0,58.0,light,10090.0,Mus musculus,NaN,NaN,IGKV4-59*01,NaN,NaN,NaN,IGKJ4*01,QIVLTQSPAIMSASPGEKVTMTCSASSSVSYMNWYQQKSGTSPKRW...,1SY6_L,NaN,QQWSSNPFT,NaN,NaN,88.0,96.0,NaN,SSVSY,NaN,NaN,27.0,31.0,NaN,DTS,NaN,NaN,49.0,51.0
3,4.0,4.0,http://www.iedb.org/reference/312,http://www.iedb.org/epitope/226962,"V: F43; W: Y71, K74, Q105, I106, M107, R108, I...",VEGF homodimer,Homo sapiens,B cell,"1708357, 1708358, 1708372, 1708373, 1708375",Fab-12 variant Y0317,NaN,NaN,heavylight,heavy,10090.0,Mus 

In [16]:
#data.to_csv(datadir / 'BCRLCDR3.csv', index=False)

In [17]:
2455

2455